In [3]:
import sqlite3
import pandas as pd

In [4]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [5]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:

data.to_sql('boston', con, index=False, if_exists='replace')

506

In [7]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 7
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [8]:
pd.read_sql(
    '''
    SELECT 
        count(*) - count (CRIM) as CRIM_cnt_null,
        count(*) - count (ZN) as ZN_cnt_null,
        count(*) - count (INDUS) as INDUS_cnt_null,
        count(*) - count (CHAS) as CHAS_cnt_null,
        count(*) - count (NOX) as NOX_cnt_null
        
    FROM boston
    
    ''',
    con,
)

,CRIM_cnt_null,ZN_cnt_null,INDUS_cnt_null,CHAS_cnt_null,NOX_cnt_null
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [9]:
pd.read_sql(
    '''
    SELECT 
        count (DISTINCT CRIM) as CRIM_cnt,
        count (DISTINCT ZN) as ZN_cnt,
        count (DISTINCT INDUS) as INDUS_cnt,
        count (DISTINCT CHAS) as CHAS_cnt,
        count (DISTINCT NOX) as NOX_cnt
        
    FROM boston
    
    ''',
    con,
)

,CRIM_cnt,ZN_cnt,INDUS_cnt,CHAS_cnt,NOX_cnt
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.

In [10]:
pd.read_sql(
    '''
    SELECT *
        
    FROM (
    SELECT 'Med_CRIM' AS column, Med_CRIM AS value, (select min(CRIM) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1))

    UNION ALL
    SELECT 'Med_ZN' AS column, Med_ZN AS value, (select min(ZN) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1))

    UNION ALL
    SELECT 'Med_INDUS' AS column, Med_INDUS AS value, (select min(INDUS) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1))

    UNION ALL
    SELECT 'Med_CHAS' AS column, Med_CHAS AS value, (select min(CHAS) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1))

    UNION ALL
    SELECT 'Med_NOX' AS column, Med_NOX AS value, (select min(NOX) from boston) AS minsd FROM (
    SELECT
        AVG(CRIM) as Med_CRIM,
        AVG(ZN) as Med_ZN,
        AVG(INDUS) as Med_INDUS,
        AVG(CHAS) as Med_CHAS,
        AVG(NOX) as Med_NOX
    FROM (SELECT *  
    FROM boston
    LIMIT 2 OFFSET (SELECT COUNT(*) FROM boston) / 2 - 1)))
    WHERE value = minsd
    
    ''',
    con,
)


,column,value,minsd
0,Med_CHAS,0.0,0.0


объяснение того, что медиана схожа с мин.значением в том, что мало мест с границами у реки и среди большого числа без реки и выпала медиана.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами.

Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [96]:
pd.read_sql(
    '''
    SELECT 'R_25' as NAME, (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV desc
    limit 25)) 
    - (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV 
    limit 25)) AS AVG
    union
    SELECT 'R_50' as NAME, (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV desc
    limit 50)) 
    - (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV 
    limit 50)) AS AVG
    union
    SELECT 'R_100' as NAME, (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV desc
    limit 100)) 
    - (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV 
    limit 100)) AS AVG
    union
    SELECT 'R_200' as NAME, (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV desc
    limit 200)) 
    - (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV 
    limit 200)) AS AVG
    union
    SELECT 'R_300' as NAME, (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV desc
    limit 300)) 
    - (SELECT avg(RM)
    FROM (SELECT 
        RM
    FROM boston
    ORDER BY MEDV 
    limit 300)) AS AVG
    ''',
    con,
)

,NAME,AVG
0,R_100,1.311640
1,R_200,0.846975
2,R_25,1.889480
3,R_300,0.566780
4,R_50,1.733820


с увеличением количества выборки средние число уменьшается и это объясняется тем, что чем больше комнат тем дороже жилье и с увеличением количества выборки комнаты уменьшаются.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [94]:
pd.read_sql(
    '''
    SELECT LSTAT, MEDV, rank() OVER(order by LSTAT) as RANK_LSTAT
    FROM boston
    order by MEDV desc
    

    ''',
    con,
)

,LSTAT,MEDV,RANK_LSTAT
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
...,...,...,...
501,36.98,7.0,505
502,29.97,6.3,494
503,26.77,5.6,480
504,22.98,5.0,455


вывод: людей с низким доходом больше

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [111]:
pd.read_sql(
    '''
    SELECT 'ГРАНИЧИТ' AS RIVER, (
    SELECT ROUND(avg(MEDV),2)
    FROM boston
    WHERE CHAS = 1) AS AVG_MEDV
    UNION
    SELECT 'НЕ ГРАНИЧИТ' AS RIVER, (
    SELECT ROUND(avg(MEDV),2) AS AVG_MEDV
    FROM boston
    WHERE CHAS = 0) AS AVG_MEDV
    
    
    ''',
    con,
)

,RIVER,AVG_MEDV
0,ГРАНИЧИТ,28.44
1,НЕ ГРАНИЧИТ,22.09


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [116]:
pd.read_sql(
    '''
    
    SELECT AVG(CRIM), AVG(ZN), AVG(INDUS),  AVG(NOX)
    FROM boston
    WHERE CHAS = 1
    
    
    
    ''',
    con,
)

,AVG(CRIM),AVG(ZN),AVG(INDUS),AVG(NOX)
0,1.85167,7.714286,12.719143,0.593426


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [144]:
pd.read_sql(
    '''
    SELECT INDUS, NOX,
        PERCENTILE_CONT(0.1) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_10,
        PERCENTILE_CONT(0.2) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_20,
        PERCENTILE_CONT(0.3) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_30,
        PERCENTILE_CONT(0.4) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_40,
        PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_50,
        PERCENTILE_CONT(0.6) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_60,
        PERCENTILE_CONT(0.7) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_70,
        PERCENTILE_CONT(0.8) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_80,
        PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_90,
        PERCENTILE_CONT(1) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_100,
        PERCENTILE_CONT(0.1) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_10,
        PERCENTILE_CONT(0.2) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_20,
        PERCENTILE_CONT(0.3) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_30,
        PERCENTILE_CONT(0.4) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_40,
        PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_50,
        PERCENTILE_CONT(0.6) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_60,
        PERCENTILE_CONT(0.7) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_70,
        PERCENTILE_CONT(0.8) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_80,
        PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_90,
        PERCENTILE_CONT(1) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_100
    FROM boston
    WHERE CHAS = 1
    --WHERE CRIM IS NULL
    ''',
    con,
)

DatabaseError: Execution failed on sql '
    SELECT INDUS, NOX,
        PERCENTILE_CONT(0.1) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_10,
        PERCENTILE_CONT(0.2) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_20,
        PERCENTILE_CONT(0.3) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_30,
        PERCENTILE_CONT(0.4) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_40,
        PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_50,
        PERCENTILE_CONT(0.6) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_60,
        PERCENTILE_CONT(0.7) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_70,
        PERCENTILE_CONT(0.8) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_80,
        PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_90,
        PERCENTILE_CONT(1) WITHIN GROUP(ORDER BY INDUS) OVER() AS PRCT_INDUS_100,
        PERCENTILE_CONT(0.1) WITHIN GROUP(ORDER BY NOX) OVER() AS PRCT_NOX_10,
        PERCENTILE_CONT(0.2)

In [8]:
pd.read_sql(
    '''
    SELECT 
    PERCENTILE_CONT(0.1) WITHIN GROUP(ORDER BY CRIM) OVER(PARTITION BY CRIM) AS PRCT_CRIM
          --AVG(ZN), AVG(INDUS),  AVG(NOX)
    FROM boston
    WHERE CHAS = 1
    
    ''',
    con,
)

DatabaseError: Execution failed on sql '
    SELECT 
    PERCENTILE_CONT(0.1) WITHIN GROUP(ORDER BY CRIM) OVER(PARTITION BY CRIM) AS PRCT_CRIM
          --AVG(ZN), AVG(INDUS),  AVG(NOX)
    FROM boston
    WHERE CHAS = 1
    
    ': near "(": syntax error